<a href="https://colab.research.google.com/github/trianadia/statistic/blob/main/Tugas8EstimasiParameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("top2022.csv")

target = 'Nilai.Total'
features = [c for c in ['Gap','Provinsi','Kab.Kota','Jenis','Ranking'] if c in df.columns]

Y = df[[target]].copy()
X_raw = df[features].copy()

for c in X_raw.columns:
    if np.issubdtype(X_raw[c].dtype, np.number):
        X_raw[c] = X_raw[c].fillna(X_raw[c].mean())
    else:
        X_raw[c] = X_raw[c].fillna("MISSING")

X = pd.get_dummies(X_raw, drop_first=True)
X.insert(0, 'Intercept', 1)

X_mat = X.values.astype(float)
Y_mat = Y.values.astype(float)

def TRANS(A):
    return A.T

def MULT(A, B):
    return A.dot(B)

def INVERS(A):
    try:
        det = np.linalg.det(A)
    except:
        det = 0.0
    if abs(det) < 1e-12:
        return np.linalg.pinv(A)
    return np.linalg.inv(A)

XT = TRANS(X_mat)
XtX = MULT(XT, X_mat)
XtX_invers = INVERS(XtX)
XtY = MULT(XT, Y_mat)
beta = MULT(XtX_invers, XtY)

beta_df = pd.DataFrame(beta, index=X.columns, columns=['beta'])
print(beta_df)

y_hat = MULT(X_mat, beta)
resid = Y_mat - y_hat
SSE = (resid**2).sum()
SSY = ((Y_mat - Y_mat.mean())**2).sum()
MSE = SSE / (len(Y_mat) - X_mat.shape[1]) if (len(Y_mat) - X_mat.shape[1]) > 0 else np.nan
R2 = 1 - SSE/SSY if SSY != 0 else np.nan

print("SSE:", SSE)
print("MSE:", MSE)
print("R2 :", R2)


                                beta
Intercept                 581.523826
Gap                        -0.016006
Ranking                    -0.083160
Provinsi_BALI               3.606676
Provinsi_BANTEN             5.218993
...                              ...
Kab.Kota_KOTA TEGAL        -5.061112
Kab.Kota_KOTA YOGYAKARTA    5.451958
Jenis_Paket C             -13.917473
Jenis_SMA                  -3.961120
Jenis_SMK                  -0.797333

[220 rows x 1 columns]
SSE: 104785.16315430882
MSE: 134.33995276193437
R2 : 0.8472125335126908
